## AI Tools Final Project Baseline Model: kNN Fake News Classifier

**Basic kNN model that predicts if articles are real or fake news by taking in the headlines that are vectorized using 'Bag of Words' approach. Simple word count is used for vectors, no weighting by frequency or sequential data included.**

In [1]:
import pandas as pd
import numpy as np
import re

**Import Dataset: "Fake News Classification". We only want the headline and the label.**

In [2]:
data = pd.read_csv("WELFake_Dataset.csv")
data = data.dropna()
data = data.drop(columns = ["text"])
data = data.drop(columns = ["Unnamed: 0"])
data.head(10)

FileNotFoundError: [Errno 2] No such file or directory: 'WELFake_Dataset.csv'

**Subset of data being used.**

In [3]:
data = data

NameError: name 'data' is not defined

**Method that takes in one headline, preprocesses it into a list of words, and adds all the new, unique words into a global frequency dictionary for the entire dataset.**

In [5]:
def make_frequency_dict(headline):
    words_list = process_sentence(headline)

    for word in words_list:
        if word in frequency_dict:
            frequency_dict[word] += 1
        else:
            frequency_dict[word] = 1

**Preprocessing method that takes in text and puts it into a list of individual, lowercase words with no numbers or punctuation.**

In [6]:
def process_sentence(text):
    text = re.sub(r'\d', '', text)
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower()
    words_list = text.split()
    return words_list

**Method that takes in all the headlines of the entire dataset and the total unique words list and returns a 2D numpy array that contains a vector for each headline. Each vector is total_unique_words in length and each index in the vector represents the count of that word in that specific headline.**

In [7]:
def headline_to_vector_fast(headlines, total_unique_words):
    # processes all headlines at once
    word_lists = headlines.apply(process_sentence)

    # create a dictionary of words as keys, indices as values
    word_to_index = {word: idx for idx, word in enumerate(total_unique_words)}

    # create 2D numpy array of zeros, specify integers in dictionary
    vectors = np.zeros((len(headlines), len(total_unique_words)), dtype=int)

    # creates array with each headline having a vector 
    for i, words in enumerate(word_lists):
        for word in words:
            if word in word_to_index:
                vectors[i, word_to_index[word]] += 1

    return vectors

**Runs the above methods to fully preprocess the dataset into vectors that can be put into the kNN model.**

In [9]:
# create frequency dictionary 
frequency_dict = {}
titles_list = data["title"].tolist()
for title in titles_list:
    make_frequency_dict(title)

# extract list of unique words
total_unique_words = list(frequency_dict.keys())
size = len(total_unique_words)
print("Number of unique words:", size)

# create input array of dimension: number headlines by unique word vector
X = headline_to_vector_fast(data["title"], total_unique_words)

print(X.shape)
print("X is", type(X))

NameError: name 'data' is not defined

**Basic kNN model used in class.**

In [10]:
import random
import numpy as np
from scipy.spatial import distance
from scipy import stats
import math
from statistics import mode

class KNN:
    """
    Implementation of the k-nearest neighbors algorithm for classification.
    """
    def __init__(self, k):
        """
        Takes one parameter.  k is the number of nearest neighbors to use
        to predict the output variable's value for a query point.
        """
        self.k = k

    def fit(self, X, y):#takes in two lists, one of x value and one of y, to create data to check future query points against
        """
        Stores the reference points (X) and their known output values (y).
        """
        self.X = X #X is a list of points that are your value
        self.y = y #Y is a list of those points output

    def predict_loop(self, X_test): #takes in a test list of x values, needs to caclualte the y values
        """
        Predicts the output variable's values for the query points X using loops.

        """
        Y = []
        for xVar in X_test:
            xDistanceDict = {}
            index = 0;
            for xVar2 in self.X:

                distance = 0
                for i in range(len(xVar2)):
                    distance += ((xVar2[i] - xVar[i]) ** 2)

                xDistanceDict[math.sqrt(distance)] = index
                index += 1

            sortedDict = {}
            sortedKeys = sorted(xDistanceDict.keys())
            for x in sortedKeys:
                sortedDict[x] = xDistanceDict[x]

            kValues = sortedKeys[:self.k]
            yArray = np.asarray(self.y)
            yResultList = []
            for key in kValues:
                yResultList.append(yArray[sortedDict[key]])

            Y.append(mode(yResultList))

        return np.asarray(Y)

**runKNN method that creates the model, 'fits' the data, predicts the testing set, and returns the accuracy of the testing set.**

In [12]:
from sklearn.metrics import accuracy_score

def runKNN(kVal, train_x, train_y, test_x, test_y):
    knn = KNN(kVal)
    knn.fit(train_x, train_y)
    predictions = knn.predict_loop(test_x)
    return accuracy_score(test_y, predictions)

**Train/test split using the preprocessed 2D numy array created earlier and the labels form the dataset. 80/20 train/test split is standard.**

In [13]:
from sklearn.model_selection import train_test_split
labels_vector = data["label"].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, labels_vector, test_size=0.2, random_state=0)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

NameError: name 'data' is not defined

In [14]:
# runKNN(1, X_train, y_train, X_test, y_test)

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

NameError: name 'X_train' is not defined